In [1]:
import os
import json
import random
from pathlib import Path
from zipfile import ZipFile
from PIL import Image

In [2]:
import kaggle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from classification_models.tfkeras import Classifiers

In [3]:
import torch
from collections import Counter
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
import os
from glob import glob
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Data science tools
import numpy as np
import pandas as pd
import os

# Image manipulations
from PIL import Image

# from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 14

In [4]:
kaggle_data = json.load(open(str(Path.home()) + '/kaggle.json', 'r'))
KAGGLE_USERNAME = kaggle_data['username']
KAGGLE_KEY = kaggle_data['key']

In [5]:
if KAGGLE_KEY and KAGGLE_USERNAME: print('Successfully imported access to kaggle')

Successfully imported access to kaggle


In [6]:
os.environ['KAGGLE_USERNAME'] = KAGGLE_USERNAME
os.environ['KAGGLE_KEY'] = KAGGLE_KEY

In [7]:
!rm -rf *.zip

In [8]:
print('Dataset will be downloaded as zip under the current directory')
!kaggle datasets download -d aalborguniversity/sewerpointclouds

Dataset will be downloaded as zip under the current directory
 96%|███████████████████████████████████████▏ | 177M/185M [00:01<00:00, 119MB/s]
100%|█████████████████████████████████████████| 185M/185M [00:01<00:00, 114MB/s]


In [9]:
!ls

 data
'fastai - multilabel classification.ipynb'
'from scratch - kaggle dataset download.ipynb'
 outputs
'pytorch - multiclass - data exploration - 1 - boats.ipynb'
'pytorch - multiclass - data exploration - 2 - shape.ipynb'
'pytorch - multiclass - data exploration - 3 - stanford cars.ipynb'
'pytorch - multiclass - model creation - 1 - boats.ipynb'
'pytorch - multiclass - model creation - 2 - shape.ipynb'
'pytorch - multiclass - model creation - 3 - stanford cars.ipynb'
 resnet50-transfer.pt
 resnet50-transfer.pth
 sewerpointclouds.zip


In [10]:
!rm -rf images/

In [11]:
!unzip *.zip -d images/

Archive:  sewerpointclouds.zip
  inflating: images/data_loading.py  
  inflating: images/testing_pointcloud_hdf5_real.h5  
  inflating: images/testing_pointcloud_hdf5_synthetic.h5  
  inflating: images/training_pointcloud_hdf5_real.h5  
  inflating: images/training_pointcloud_hdf5_synthetic.h5  


In [22]:
def print_lidar_data():
    import os
    import h5py
    import numpy as np


    dataDir = "."
    hdf5Files = ["training_pointcloud_hdf5", "testing_pointcloud_hdf5"]
    dataTypes = ["synthetic", "real"]
    partitions = ["Training", "Validation"]

    classLabels = {0:"Normal", 1:"Displacement", 2:"Brick", 3:"Rubber Ring"}


    for h5 in hdf5Files:
        for dt in dataTypes:
            path = os.path.join(dataDir, "{}_{}.h5".format('images/'+ h5, dt))

            with h5py.File(path, 'r') as hdf:          
                if h5 == "training_pointcloud_hdf5":
                    partitions = ["Training", "Validation"]
                else:
                    partitions = ["Testing"]

                for partition in partitions:
                    data = np.asarray(hdf[f'{partition}/PointClouds'][:])
                    labels = np.asarray(hdf[f'{partition}/Labels'][:])
                    uniqueLabels, uniqueCounts = np.unique(labels, return_counts = True)

                    print(f'\nFilepath: {path}')
                    print(f'[{partition} Data]')
                    print(f'Data Shape: {data.shape} | Type: {data[0].dtype}')
                    print(f'Label Shape: {labels.shape} | Type: {labels[0].dtype}')
                    print(f'Labels: {uniqueLabels} | Counts: {uniqueCounts}\n')
                    
print_lidar_data()


Filepath: ./images/training_pointcloud_hdf5_synthetic.h5
[Training Data]
Data Shape: (10800, 1024, 3) | Type: float32
Label Shape: (10800,) | Type: int64
Labels: [0 1 2 3] | Counts: [5365 1811 1822 1802]


Filepath: ./images/training_pointcloud_hdf5_synthetic.h5
[Validation Data]
Data Shape: (2700, 1024, 3) | Type: float32
Label Shape: (2700,) | Type: int64
Labels: [0 1 2 3] | Counts: [1385  439  428  448]


Filepath: ./images/training_pointcloud_hdf5_real.h5
[Training Data]
Data Shape: (274, 1024, 3) | Type: float32
Label Shape: (274,) | Type: int64
Labels: [0 1 2 3] | Counts: [140  45  45  44]


Filepath: ./images/training_pointcloud_hdf5_real.h5
[Validation Data]
Data Shape: (68, 1024, 3) | Type: float32
Label Shape: (68,) | Type: int64
Labels: [0 1 2 3] | Counts: [31 12 12 13]


Filepath: ./images/testing_pointcloud_hdf5_synthetic.h5
[Testing Data]
Data Shape: (2700, 1024, 3) | Type: float32
Label Shape: (2700,) | Type: int64
Labels: [0 1 2 3] | Counts: [1350  450  450  450]


Fil

In [12]:
!pip install tables

In [13]:
from pandas import HDFStore

In [32]:
hdf.close()

In [33]:
hdf = HDFStore('images/training_pointcloud_hdf5_real.h5'); hdf

<class 'pandas.io.pytables.HDFStore'>
File path: images/training_pointcloud_hdf5_real.h5

In [37]:
hdf = HDFStore('images/testing_pointcloud_hdf5_real.h5'); hdf

<class 'pandas.io.pytables.HDFStore'>
File path: images/testing_pointcloud_hdf5_real.h5

In [38]:
hdf.keys()

[]

In [39]:
list(hdf.items())

[]

In [36]:
hdf.get_storer('Training Data').table

KeyError: 'No object named Training Data in the file'

In [21]:
hdf.get_storer('Normal').table

KeyError: 'No object named Normal in the file'

In [ ]:
truc = pd.read_hdf('images/training_pointcloud_hdf5_real.h5')
truc

In [ ]:
truc = pd.read_hdf('images/training_pointcloud_hdf5_synthetic.h5')
truc

In [ ]:
ImagePATH = f"images/cars_train/"
ImagePATH

In [ ]:
categories = glob(ImagePATH+"*")
categories

In [ ]:
ImagePATH = f"images/cars_test/"
ImagePATH

In [ ]:
len(categories)

In [ ]:
filedf = pd.DataFrame()
for cat in categories:
    files = glob(cat+"/*")
    print(files)
    tempdf = pd.DataFrame({ 'filepath':files, 'category':cat.split("/")[-1] })
    print(tempdf)
    filedf = pd.concat([filedf,tempdf])
filedf.shape

In [ ]:
filedf.head()

In [ ]:
gby_cnt = filedf.groupby("category").aggregate('count').rename(columns = {'filepath':'cnt'}).reset_index().sort_values(by='cnt',ascending=False)

In [ ]:
px.bar(gby_cnt, x='category', y='cnt', color='category', title='Counts from Each Category')

In [ ]:
!mkdir outputs

In [ ]:
gby_cnt.to_csv(f"outputs/category_counts.csv", index=None)

## Img exploration

In [ ]:
plt.figure(figsize=(24, 24))

for i in range(16):
    path = filedf.sample(1)['filepath'].values[0]
    category = path.split("/")[1]
    ex_img = Image.open(path)
    ax = plt.subplot(4, 4, i + 1)
    ax.annotate(category, xy=(0.65, 0.9), xycoords="axes fraction",weight='bold',size=20)
    ax.imshow(ex_img)

plt.tight_layout()

In [ ]:
focus_cats = gby_cnt[gby_cnt['cnt'] >=40]['category'].values

In [ ]:
filedf

In [ ]:
filedf = filedf[filedf['category'].isin(focus_cats)]

In [ ]:
filedf.category.value_counts()

## Splitting dataset

In [ ]:
X_train, X_test, _, _ = train_test_split(
        filedf, filedf['category'],stratify=filedf['category'], test_size=0.4)

X_test, X_val, _, _ = train_test_split(
        X_test, X_test['category'], stratify=X_test['category'], test_size=0.5)

In [ ]:
X_train['type'] = 'train'
X_val['type'] = 'val'
X_test['type'] = 'test'

In [ ]:
fulldf = pd.concat([X_train, X_test, X_val])
fulldf['type'].value_counts()

In [ ]:
fulldf.head()

In [ ]:
# Moving data to train/test/val folder
!rm -rf data
!mkdir data
!rm -rf data/
!rm -rf data/train
!rm -rf data/test
!rm -rf data/val
!mkdir data/
!mkdir data/train
!mkdir data/test
!mkdir data/val

In [ ]:
for cat in fulldf.category.unique():
    os.system(f"mkdir data/train/'{cat}'") 
    os.system(f"mkdir data/test/'{cat}'") 
    os.system(f"mkdir data/val/'{cat}'")

In [ ]:
fulldf.head()

In [ ]:
for i,row in fulldf.iterrows():
    # Boat category
    cat = row['category']
    # section is train,val or test
    section = row['type']
    # input filepath to copy
    ipath = row['filepath']
    # output filepath to pastes
    opath = ipath.replace(f"{ImagePATH}", f"data/{section}/")
    # running the cp command
    os.system(f"cp '{ipath}' '{opath}'")

In [ ]:
fulldf